<a href="https://colab.research.google.com/github/abeguly/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-13 23:04:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-02-13 23:04:48 (6.61 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({'customer_id':'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   39290568|             1|
|   23601866|             1|
|   19004376|             1|
|   16320823|             1|
|   13973119|             1|
|    1155154|             1|
|   24317024|             2|
|   15125275|             1|
|   35781732|             2|
|   40985731|            12|
|   19522542|             1|
|   50243180|             1|
|    2702985|             1|
|   41857916|             1|
|    1943933|             2|
|   42050930|             5|
|    4867895|             1|
|   46317087|             3|
|   52021773|             2|
|   15504622|             9|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B014PO1708|       See You Again|
|B001APIS9Y|What Happened to ...|
|B005XT4CAW|I Can Dream About...|
|B006WH9FPQ|  If You Want To Fly|
|B0026G2MG6|Ghetto Fabolous [...|
|B0096RUQZW|       Call Me Maybe|
|B003YZGVFQ|Maxwell's silver ...|
|B00138B7N2|He Touched Me (Al...|
|B00OLF0U3M|Beyond The Lights...|
|B007D89P76|A Hampshire Suite...|
|B0011U5E1S|             Nirvana|
|B00JWFMVNS|Sousa (Red Classics)|
|B00120EDBY|Talkin' To You Ey...|
|B00IWI3HNO|          Sweet Life|
|B00YIB4KJY|                Fall|
|B0011ZVFIO|Carmel City (LP V...|
|B00C32RB4Q|         Lola Montez|
|B001NTZ4NA|Thug Lovin' (Albu...|
|B001BHJZN4|      Eye In The Sky|
|B005SR8QO2|      My Kinda Party|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R14LVB34Z2Z53I|   10293599|B000YMOQZY|     262935067| 2015-08-31|
|R2M0R9CWPC82LT|    6087195|B00ISAEC80|     452318038| 2015-08-31|
| RQNQI2R3BM52A|   33717063|B00VBB6AVU|     675969129| 2015-08-31|
| RTUIX1135HUAB|   14948020|B008A5POJO|     282925893| 2015-08-31|
| RE3QSAT8TL010|   21694522|B014CW2RN4|     109077793| 2015-08-31|
|R14TN65IY0NUOD|   14948020|B00DRA3EOY|     959245080| 2015-08-31|
|R3LIF8MCNH1ODF|   20031966|B009G3U0PW|     762436916| 2015-08-31|
|R33X0DGA4PPQ3L|   12223745|B013XBYQOS|     137980344| 2015-08-31|
|R2DJJP7DQRT1ZW|   30231876|B003L5H25W|     165995375| 2015-08-31|
|R2A8XDXW5XOT4H|   51714365|B00136NVH4|     227508666| 2015-08-31|
|R3MJK3L7ER61TT|   37216001|B0030AKPQ6|     164096943| 2015-08-31|
|R38Z7XMCVDR0R5|   49207156|B014EQE0V0|     714368268| 2015-08

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14LVB34Z2Z53I|          5|            0|          0|   N|                Y|
|R2M0R9CWPC82LT|          5|            0|          0|   N|                Y|
| RQNQI2R3BM52A|          4|            0|          0|   N|                Y|
| RTUIX1135HUAB|          5|            0|          0|   N|                Y|
| RE3QSAT8TL010|          4|            0|          0|   N|                Y|
|R14TN65IY0NUOD|          5|            0|          0|   N|                Y|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|                Y|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|                N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|                Y|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cqc7tm0ifgyo.us-east-1.rds.amazonaws.com:5432/Challenge16"
config = {"user":"postgres", 
          "password": "XXX", 
          "driver":"org.postgresql.Driver"}

In [15]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [17]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [18]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# Deliverable 2




In [31]:
# filder data and create new dataframe where total vote => 20
total_votes20 = df.filter(df["total_votes"] >= 20)
total_votes20.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [41]:

helpful_total = total_votes20.filter(total_votes20["helpful_votes"]/total_votes20["total_votes"] >= .5)
helpful_total.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [57]:
review_df = helpful_total.filter(helpful_total['vine'] == 'Y')
review_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [58]:
no_review_df = helpful_total.filter(helpful_total['vine'] == 'N')
no_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [59]:
review_df.count()

0

In [60]:

review_df.filter(review_df['star_rating'] == 5).count()

0

In [63]:
review_df['star_rating']==5 / review_df.count()

ZeroDivisionError: ignored

In [64]:
no_review_df.count()

4532

In [73]:
no_review_df.filter(no_review_df["star_rating"] == 5).count()

2507

In [74]:
no_review_df.filter(no_review_df['star_rating'] == 5).count() / no_review_df.count()

0.5531774051191527